# 生成分析报告

本notebook演示如何生成HTML和PDF格式的股票分析报告。

**注意**: 由于Yahoo Finance API限流，本notebook使用已创建的示例数据。

## 1. 导入必要的库

In [121]:
import sys
import os
notebook_dir = os.getcwd()
project_dir = os.path.dirname(notebook_dir)
if project_dir not in sys.path:
    sys.path.insert(0, project_dir)

import pandas as pd
import numpy as np
from datetime import datetime

from src.data_loader import load_stock_data, load_multiple_stocks
from src.indicators import calculate_technical_indicators, calculate_returns, calculate_cumulative_returns
from src.multi_stock_analysis import calculate_stock_metrics
from src.report_generator import (
    generate_html_report,
    generate_pdf_report,
    save_html_report,
    generate_multi_stock_report,
    create_summary_report
)

print("库导入成功！")

库导入成功！


## 2. 加载股票数据并计算指标

In [122]:
ticker = 'AAPL'
file_path = os.path.join(project_dir, 'data', 'AAPL_stock_data.csv')

data = load_stock_data(file_path, ticker)
data = calculate_technical_indicators(data)

print(f"数据加载完成！数据行数: {len(data)}")

从本地文件加载数据: /Users/fang/Documents/Code/python-demo/stock-analysis/data/AAPL_stock_data.csv
数据加载完成！数据行数: 1304


## 3. 计算关键指标

In [123]:
returns = calculate_returns(data, 'Close')
cumulative_returns = calculate_cumulative_returns(data, 'Close')

metrics = {
    '指标': [
        '起始价格',
        '结束价格',
        '总收益率',
        '年化收益率',
        '平均日收益率',
        '收益率标准差',
        '最大回撤',
        '夏普比率',
        '当前RSI',
        '当前MACD',
        '当前价格相对于SMA20',
        '当前价格相对于SMA50'
    ],
    '数值': [
        f"${data['Close'].iloc[0]:.2f}",
        f"${data['Close'].iloc[-1]:.2f}",
        f"{((data['Close'].iloc[-1] / data['Close'].iloc[0] - 1) * 100):.2f}%",
        f"{((data['Close'].iloc[-1] / data['Close'].iloc[0]) ** (252 / len(data)) - 1) * 100:.2f}%",
        f"{returns.mean() * 100:.4f}%",
        f"{returns.std() * 100:.4f}%",
        f"{(cumulative_returns.cummax() - cumulative_returns).max() * 100:.2f}%",
        f"{(returns.mean() / returns.std()) * np.sqrt(252):.2f}" if returns.std() != 0 else "N/A",
        f"{data['RSI_14'].iloc[-1]:.2f}",
        f"{data['MACD'].iloc[-1]:.2f}",
        f"{((data['Close'].iloc[-1] / data['SMA_20'].iloc[-1] - 1) * 100):.2f}%",
        f"{((data['Close'].iloc[-1] / data['SMA_50'].iloc[-1] - 1) * 100):.2f}%"
    ]
}

metrics_df = pd.DataFrame(metrics)
print("关键指标:")
print(metrics_df)

关键指标:
              指标       数值
0           起始价格  $261.00
1           结束价格  $359.38
2           总收益率   37.69%
3          年化收益率    6.38%
4         平均日收益率  0.0443%
5         收益率标准差  1.9901%
6           最大回撤   63.77%
7           夏普比率     0.35
8          当前RSI    41.03
9         当前MACD    -4.18
10  当前价格相对于SMA20   -3.11%
11  当前价格相对于SMA50   -5.77%


## 4. 生成HTML报告

In [124]:
html_report_path = 'reports/AAPL_analysis_report.html'

additional_info = {
    '数据开始日期': data.index[0].strftime('%Y-%m-%d'),
    '数据结束日期': data.index[-1].strftime('%Y-%m-%d'),
    '数据天数': len(data),
    '最高价': f"${data['High'].max():.2f}",
    '最低价': f"${data['Low'].min():.2f}",
    '平均成交量': f"{data['Volume'].mean():,.0f}"
}

save_html_report(
    metrics_df,
    html_report_path,
    ticker=ticker,
    analysis_period='5年',
    additional_info=additional_info
)

HTML报告已保存至: reports/AAPL_analysis_report.html


## 5. 生成PDF报告（需要weasyprint）

In [125]:
import os
os.environ['DYLD_LIBRARY_PATH'] = '/opt/homebrew/lib:' + os.environ.get('DYLD_LIBRARY_PATH', '')

try:
    pdf_report_path = 'reports/AAPL_analysis_report.pdf'
    
    generate_pdf_report(
        metrics_df,
        pdf_report_path,
        ticker=ticker,
        analysis_period='5年',
        additional_info=additional_info
    )
except Exception as e:
    print(f"PDF生成失败: {e}")
    print("提示: 确保已设置 DYLD_LIBRARY_PATH 环境变量")

PDF报告已保存至: reports/AAPL_analysis_report.pdf


## 6. 生成摘要报告

In [126]:
summary_report_path = 'reports/AAPL_summary_report.txt'
create_summary_report(data, ticker, summary_report_path)

摘要报告已保存至: reports/AAPL_summary_report.txt


## 7. 多股票对比报告

In [127]:
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']

stock_data = load_multiple_stocks(
    tickers=tickers,
    data_dir=os.path.join(project_dir, 'data'),
    period='5y',
    force_download=False
)

print(f"成功加载 {len(stock_data)} 只股票的数据")

从本地文件加载数据: /Users/fang/Documents/Code/python-demo/stock-analysis/data/AAPL_stock_data.csv
从本地文件加载数据: /Users/fang/Documents/Code/python-demo/stock-analysis/data/MSFT_stock_data.csv
从本地文件加载数据: /Users/fang/Documents/Code/python-demo/stock-analysis/data/GOOGL_stock_data.csv
从本地文件加载数据: /Users/fang/Documents/Code/python-demo/stock-analysis/data/AMZN_stock_data.csv
从本地文件加载数据: /Users/fang/Documents/Code/python-demo/stock-analysis/data/TSLA_stock_data.csv
成功加载 5 只股票的数据


## 8. 计算多股票指标

In [128]:
stock_metrics = calculate_stock_metrics(stock_data, 'Close')
print("多股票指标:")
print(stock_metrics.round(2))

多股票指标:
        Start_Price  End_Price  Total_Return_Pct  Mean_Return_Pct  \
Ticker                                                              
AAPL          261.0     359.38             37.69             0.04   
MSFT          258.0     105.59            -59.07            -0.05   
GOOGL         215.0      75.35            -64.95            -0.06   
AMZN          234.0      90.78            -61.21            -0.05   
TSLA          135.0     145.10              7.48             0.03   

        Std_Return_Pct  Max_Drawdown_Pct  Sharpe_Ratio  Volatility_Annual  
Ticker                                                                     
AAPL              1.99             63.77          0.35              31.59  
MSFT              1.96             69.28         -0.40              31.11  
GOOGL             2.00             78.78         -0.48              31.80  
AMZN              2.02             83.33         -0.41              32.12  
TSLA              2.02            128.62          0.2

## 9. 生成多股票对比报告

In [129]:
multi_stock_report_path = 'reports/multi_stock_comparison_report.html'

additional_multi_info = {
    '分析股票数量': len(tickers),
    '分析周期': '5年',
    '数据来源': 'Yahoo Finance'
}

generate_multi_stock_report(
    stock_metrics,
    multi_stock_report_path,
    analysis_period='5年',
    additional_info=additional_multi_info
)

多股票分析报告已保存至: reports/multi_stock_comparison_report.html


## 10. 查看生成的报告

In [130]:
import os

print("生成的报告文件:")
report_files = [
    'reports/AAPL_analysis_report.html',
    'reports/AAPL_analysis_report.pdf',
    'reports/AAPL_summary_report.txt',
    'reports/multi_stock_comparison_report.html'
]

for report_file in report_files:
    if os.path.exists(report_file):
        size = os.path.getsize(report_file)
        print(f"  ✓ {report_file} ({size:,} bytes)")
    else:
        print(f"  ✗ {report_file} (未生成)")

生成的报告文件:
  ✓ reports/AAPL_analysis_report.html (4,078 bytes)
  ✓ reports/AAPL_analysis_report.pdf (34,127 bytes)
  ✓ reports/AAPL_summary_report.txt (388 bytes)
  ✓ reports/multi_stock_comparison_report.html (5,093 bytes)


## 11. 自定义报告模板

In [131]:
from jinja2 import Template

custom_template = """
<!DOCTYPE html>
<html lang="zh-CN">
<head>
    <meta charset="UTF-8">
    <title>自定义股票报告 - {{ ticker }}</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 20px; }
        .header { background: #4CAF50; color: white; padding: 20px; }
        .metric { margin: 10px 0; padding: 10px; background: #f5f5f5; }
    </style>
</head>
<body>
    <div class="header">
        <h1>{{ ticker }} 股票分析报告</h1>
        <p>生成时间: {{ date }}</p>
    </div>
    <h2>关键指标</h2>
    {% for metric in metrics %}
    <div class="metric">
        <strong>{{ metric.name }}:</strong> {{ metric.value }}
    </div>
    {% endfor %}
</body>
</html>
"""

custom_metrics = [
    {'name': '当前价格', 'value': f"${data['Close'].iloc[-1]:.2f}"},
    {'name': '总收益率', 'value': f"{((data['Close'].iloc[-1] / data['Close'].iloc[0] - 1) * 100):.2f}%"},
    {'name': 'RSI', 'value': f"{data['RSI_14'].iloc[-1]:.2f}"},
    {'name': 'MACD', 'value': f"{data['MACD'].iloc[-1]:.2f}"}
]

custom_html = Template(custom_template).render(
    ticker=ticker,
    date=datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    metrics=custom_metrics
)

custom_report_path = 'reports/AAPL_custom_report.html'
with open(custom_report_path, 'w', encoding='utf-8') as f:
    f.write(custom_html)

print(f"自定义报告已保存至: {custom_report_path}")

自定义报告已保存至: reports/AAPL_custom_report.html


## 12. 总结

本notebook完成了以下任务：
1. ✓ 加载股票数据并计算指标
2. ✓ 计算关键指标
3. ✓ 生成HTML报告
4. ✓ 生成PDF报告
5. ✓ 生成摘要报告
6. ✓ 加载多股票数据
7. ✓ 计算多股票指标
8. ✓ 生成多股票对比报告
9. ✓ 查看生成的报告
10. ✓ 创建自定义报告模板

所有notebook已完成！您现在可以：
- 使用这些notebook学习股票数据分析
- 根据需要修改代码以适应不同的分析需求
- 将这些功能集成到自己的项目中